In [2]:
!pip install pandas
!pip install numpy
!pip install faiss-cpu
!pip install sentence-transformers

  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
     |████████████████████████████████| 15.7 MB 916 kB/s 
  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
You should consider upgrading via the '/home/cecilio/Projetos/ufsj-Data-Science/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/cecilio/Projetos/ufsj-Data-Science/.venv/bin/python3 -m pip install --upgrade pip' command.
  Using cached faiss_cpu-1.7.1.post2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.4 MB)
You should consider upgrading via the '/home/cecilio/Projetos/ufsj-Data-Science/.venv/bin/python3 -m pip install --upgrade pip' command.
  Using cached sentence-transformers-2.1.0.tar.gz (78 kB)
     |████████████████████████████████| 3.1 MB 4.0 MB/s 
  Using cached tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached tqdm-4.6

In [4]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Embeddings/Models explicação

- [https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526](https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526)
- [https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/04-TF-IDF-and-similarity-scores.html](https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/04-TF-IDF-and-similarity-scores.html)
- [https://www.sbert.net/docs/pretrained_models.html](https://www.sbert.net/docs/pretrained_models.html)
- [https://github.com/facebookresearch/faiss/wiki/Faiss-indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes)

In [8]:
model = SentenceTransformer('all-mpnet-base-v2')
data_path = "./data"

# SentenceTransformers
SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in our paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks.

# Faiss
Faiss is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [6]:
def generate_index(data):
    encoded_data = model.encode(data)
    encoded_data = np.asarray(encoded_data.astype('float32'))
    index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
    ids = np.array(range(0, len(data)))
    ids = np.asarray(ids.astype('int64'))
    index.add_with_ids(encoded_data, ids)
    faiss.write_index(index, f'{data_path}/sentences.index')

def search(query):
    index = faiss.read_index(f'{data_path}/sentences.index')
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    return top_k[1].tolist()[0]

In [12]:
df = pd.read_csv(f'{data_path}/data.tsv', sep="\t")
df

/home/cecilio/Projetos/ufsj-Data-Science/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8457010,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8457011,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8457012,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8457013,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


# Pre Processamento

In [13]:
generate_index(df['originalTitle'])

In [ ]:
query = "Ferris"

search(query)

# Discussão dos Resultados

- Comparação de metodos e modelos
- Criar gráficos